In [327]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf 
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.stats.outliers_influence import variance_inflation_factor 
from statsmodels.tools.tools import add_constant
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf 
import random as rm
import time as tm
import matplotlib.pyplot as plt
from scipy import stats
from pandas.api.types import is_string_dtype 
from pandas.api.types import is_numeric_dtype 
import sklearn.metrics as skm
import sklearn.impute as ski
from statsmodels.tools.tools import add_constant
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import mean_squared_error
import math
import scipy.stats as stats
import statistics

In [328]:
df1 = pd.read_csv('covid_by_county.csv')
df1.dropna(inplace=True)

In [329]:
df1['fips'] = df1['fips'].astype(int)

In [330]:
df1['fips'] = df1['fips'].astype(str)

# Separate into "STATE" and "COUNTY" columns
df1['STATE'] = df1['fips'].str[:2]
df1['COUNTY'] = df1['fips'].str[2:]

# Convert back to integers if needed
df1['STATE'] = df1['STATE'].astype(int)
df1['COUNTY'] = df1['COUNTY'].astype(int)
df1

,date,county,state,fips,cases,deaths,STATE,COUNTY
0,2020-01-21,Snohomish,Washington,53061,1,0,53,61
1,2020-01-22,Snohomish,Washington,53061,1,0,53,61
2,2020-01-23,Snohomish,Washington,53061,1,0,53,61
3,2020-01-24,Cook,Illinois,17031,1,0,17,31
4,2020-01-24,Snohomish,Washington,53061,1,0,53,61
...,...,...,...,...,...,...,...,...
359973,2020-07-22,Sweetwater,Wyoming,56037,201,2,56,37
359974,2020-07-22,Teton,Wyoming,56039,234,1,56,39
359975,2020-07-22,Uinta,Wyoming,56041,224,0,56,41
359976,2020-07-22,Washakie,Wyoming,56043,44,5,56,43


In [331]:
df1['date'] = pd.to_datetime(df1['date'])

df1['YEAR'] = df1['date'].dt.year
df1

,date,county,state,fips,cases,deaths,STATE,COUNTY,YEAR
0,2020-01-21,Snohomish,Washington,53061,1,0,53,61,2020
1,2020-01-22,Snohomish,Washington,53061,1,0,53,61,2020
2,2020-01-23,Snohomish,Washington,53061,1,0,53,61,2020
3,2020-01-24,Cook,Illinois,17031,1,0,17,31,2020
4,2020-01-24,Snohomish,Washington,53061,1,0,53,61,2020
...,...,...,...,...,...,...,...,...,...
359973,2020-07-22,Sweetwater,Wyoming,56037,201,2,56,37,2020
359974,2020-07-22,Teton,Wyoming,56039,234,1,56,39,2020
359975,2020-07-22,Uinta,Wyoming,56041,224,0,56,41,2020
359976,2020-07-22,Washakie,Wyoming,56043,44,5,56,43,2020


In [332]:
df2 = pd.read_csv('Census_data.csv')
##df = df.mask(df.lt(0), np.nan)
columns = df2.columns.tolist()

In [333]:
filtered_df2 = df2[df2['AGEGRP'] == 0]
filtered_df2

,SUMLEV,STATE,COUNTY,STNAME,CTYNAME,YEAR,AGEGRP,TOT_POP,TOT_MALE,TOT_FEMALE,...,HWAC_MALE,HWAC_FEMALE,HBAC_MALE,HBAC_FEMALE,HIAC_MALE,HIAC_FEMALE,HAAC_MALE,HAAC_FEMALE,HNAC_MALE,HNAC_FEMALE
0,50,1,1,Alabama,Autauga County,1,0,58802,28761,30041,...,854,734,112,97,40,30,22,24,19,11
19,50,1,1,Alabama,Autauga County,2,0,58902,28819,30083,...,861,736,118,106,44,37,25,22,22,14
38,50,1,1,Alabama,Autauga County,3,0,59210,28851,30359,...,895,803,127,98,49,36,25,18,19,12
57,50,1,1,Alabama,Autauga County,4,0,59759,29105,30654,...,902,810,132,112,47,40,23,22,22,19
76,50,1,3,Alabama,Baldwin County,1,0,231761,113388,118373,...,5256,4806,327,334,305,241,101,95,95,73
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238849,50,56,43,Wyoming,Washakie County,4,0,7719,3973,3746,...,507,483,11,10,58,48,6,8,6,2
238868,50,56,45,Wyoming,Weston County,1,0,6840,3724,3116,...,157,112,5,6,25,27,3,0,2,1
238887,50,56,45,Wyoming,Weston County,2,0,6818,3724,3094,...,162,113,5,8,23,23,5,6,3,1
238906,50,56,45,Wyoming,Weston County,3,0,6766,3729,3037,...,176,125,5,4,24,28,7,6,4,1


In [334]:
df5 = pd.read_csv('cc-est2022-agesex-all.csv')
##df = df.mask(df.lt(0), np.nan)
columns = df5.columns.tolist()

In [335]:
df5

,SUMLEV,STATE,COUNTY,STNAME,CTYNAME,YEAR,POPESTIMATE,POPEST_MALE,POPEST_FEM,UNDER5_TOT,...,AGE7579_FEM,AGE8084_TOT,AGE8084_MALE,AGE8084_FEM,AGE85PLUS_TOT,AGE85PLUS_MALE,AGE85PLUS_FEM,MEDIAN_AGE_TOT,MEDIAN_AGE_MALE,MEDIAN_AGE_FEM
0,50,1,1,Alabama,Autauga County,1,58802,28761,30041,3443,...,1055,1165,501,664,955,345,610,39.2,37.9,40.3
1,50,1,1,Alabama,Autauga County,2,58902,28819,30083,3457,...,1064,1170,501,669,958,350,608,39.1,37.9,40.3
2,50,1,1,Alabama,Autauga County,3,59210,28851,30359,3416,...,1074,1190,492,698,972,362,610,39.2,38.0,40.3
3,50,1,1,Alabama,Autauga County,4,59759,29105,30654,3460,...,1152,1225,495,730,1019,385,634,39.2,38.0,40.4
4,50,1,3,Alabama,Baldwin County,1,231761,113388,118373,12046,...,4861,5601,2542,3059,4610,1812,2798,43.7,42.5,44.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12571,50,56,43,Wyoming,Washakie County,4,7719,3973,3746,378,...,187,250,125,125,212,73,139,44.8,43.3,46.6
12572,50,56,45,Wyoming,Weston County,1,6840,3724,3116,308,...,109,175,70,105,191,73,118,43.5,42.7,44.8
12573,50,56,45,Wyoming,Weston County,2,6818,3724,3094,310,...,110,171,70,101,193,73,120,43.5,42.6,44.8
12574,50,56,45,Wyoming,Weston County,3,6766,3729,3037,302,...,115,151,66,85,184,69,115,43.9,43.1,45.3


In [336]:
merged_df25 = pd.merge(filtered_df2, df5, on=['STATE','COUNTY', 'STNAME', 'CTYNAME','YEAR'], how='inner')
columns_to_drop = ['AGEGRP', 'SUMLEV_x']

merged_df25.drop(columns=columns_to_drop, inplace=True)
merged_df25

,STATE,COUNTY,STNAME,CTYNAME,YEAR,TOT_POP,TOT_MALE,TOT_FEMALE,WA_MALE,WA_FEMALE,...,AGE7579_FEM,AGE8084_TOT,AGE8084_MALE,AGE8084_FEM,AGE85PLUS_TOT,AGE85PLUS_MALE,AGE85PLUS_FEM,MEDIAN_AGE_TOT,MEDIAN_AGE_MALE,MEDIAN_AGE_FEM
0,1,1,Alabama,Autauga County,1,58802,28761,30041,22036,22601,...,1055,1165,501,664,955,345,610,39.2,37.9,40.3
1,1,1,Alabama,Autauga County,2,58902,28819,30083,22028,22594,...,1064,1170,501,669,958,350,608,39.1,37.9,40.3
2,1,1,Alabama,Autauga County,3,59210,28851,30359,21910,22612,...,1074,1190,492,698,972,362,610,39.2,38.0,40.3
3,1,1,Alabama,Autauga County,4,59759,29105,30654,22026,22671,...,1152,1225,495,730,1019,385,634,39.2,38.0,40.4
4,1,3,Alabama,Baldwin County,1,231761,113388,118373,99054,103205,...,4861,5601,2542,3059,4610,1812,2798,43.7,42.5,44.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12571,56,43,Wyoming,Washakie County,4,7719,3973,3746,3755,3534,...,187,250,125,125,212,73,139,44.8,43.3,46.6
12572,56,45,Wyoming,Weston County,1,6840,3724,3116,3476,2897,...,109,175,70,105,191,73,118,43.5,42.7,44.8
12573,56,45,Wyoming,Weston County,2,6818,3724,3094,3470,2876,...,110,171,70,101,193,73,120,43.5,42.6,44.8
12574,56,45,Wyoming,Weston County,3,6766,3729,3037,3468,2822,...,115,151,66,85,184,69,115,43.9,43.1,45.3


In [337]:
df3 = pd.read_csv('GDP_by_county.csv')
df3

,County,GeoName,LineCode,Description,2020,2021,2022
0,1001,"Autauga, AL",3,Current-dollar GDP (thousands of current dollars),1813553,1947622,2364891
1,1003,"Baldwin, AL",3,Current-dollar GDP (thousands of current dollars),8762106,9879022,10811119
2,1005,"Barbour, AL",3,Current-dollar GDP (thousands of current dollars),786529,862519,930064
3,1007,"Bibb, AL",3,Current-dollar GDP (thousands of current dollars),501320,528278,558848
4,1009,"Blount, AL",3,Current-dollar GDP (thousands of current dollars),973414,1153245,1282006
...,...,...,...,...,...,...,...
3113,56037,"Sweetwater, WY",3,Current-dollar GDP (thousands of current dollars),3399440,3814348,4118875
3114,56039,"Teton, WY",3,Current-dollar GDP (thousands of current dollars),2819873,3947580,4156308
3115,56041,"Uinta, WY",3,Current-dollar GDP (thousands of current dollars),823225,913936,1033918
3116,56043,"Washakie, WY",3,Current-dollar GDP (thousands of current dollars),365071,391162,421190


In [338]:
df4 = pd.read_csv('personal_income_by_county.csv')
df4

,GeoFips,GeoName,LineCode,Description,2020,2021,2022
0,1001,"Autauga, AL",3,Per capita personal income (dollars) 2,45151,48914,49391
1,1003,"Baldwin, AL",3,Per capita personal income (dollars) 2,51230,55865,56747
2,1005,"Barbour, AL",3,Per capita personal income (dollars) 2,37111,40795,40560
3,1007,"Bibb, AL",3,Per capita personal income (dollars) 2,34938,37175,37513
4,1009,"Blount, AL",3,Per capita personal income (dollars) 2,38133,42852,43744
...,...,...,...,...,...,...,...
3135,56037,"Sweetwater, WY",3,Per capita personal income (dollars) 2,54708,56252,58374
3136,56039,"Teton, WY",3,Per capita personal income (dollars) 2,300665,362522,406054
3137,56041,"Uinta, WY",3,Per capita personal income (dollars) 2,42289,44358,44775
3138,56043,"Washakie, WY",3,Per capita personal income (dollars) 2,55098,54898,55288


In [339]:
merged_df34 = pd.merge(df3, df4, on='GeoName', how='inner')


In [340]:
columns_to_drop = ['LineCode_x', 'Description_x','GeoFips','LineCode_y','Description_y']

merged_df34.drop(columns=columns_to_drop, inplace=True)
merged_df34

,County,GeoName,2020_x,2021_x,2022_x,2020_y,2021_y,2022_y
0,1001,"Autauga, AL",1813553,1947622,2364891,45151,48914,49391
1,1003,"Baldwin, AL",8762106,9879022,10811119,51230,55865,56747
2,1005,"Barbour, AL",786529,862519,930064,37111,40795,40560
3,1007,"Bibb, AL",501320,528278,558848,34938,37175,37513
4,1009,"Blount, AL",973414,1153245,1282006,38133,42852,43744
...,...,...,...,...,...,...,...,...
3095,56037,"Sweetwater, WY",3399440,3814348,4118875,54708,56252,58374
3096,56039,"Teton, WY",2819873,3947580,4156308,300665,362522,406054
3097,56041,"Uinta, WY",823225,913936,1033918,42289,44358,44775
3098,56043,"Washakie, WY",365071,391162,421190,55098,54898,55288


In [341]:
first_two_columns = merged_df34.iloc[:, :2]
year2020_GDP = merged_df34.iloc[:, 2]
year2021_GDP = merged_df34.iloc[:, 3]
year2022_GDP = merged_df34.iloc[:, 4]
year2020_PER = merged_df34.iloc[:, 5]
year2021_PER = merged_df34y.iloc[:, 6]
year2022_PER = merged_df34.iloc[:, 7]

In [342]:
series_2020_GDP = pd.Series(year2020_GDP, name ='GDP_current_dollar')
first_two_columns['GDP_current_dollar'] = series_2020_GDP

series_2020_PER = pd.Series(year2020_PER, name ='personal_income')
first_two_columns['personal_income'] = series_2020_PER

first_two_columns['year'] = 2020

first_two_columns_2020 = first_two_columns.copy()
first_two_columns_2020

,County,GeoName,GDP_current_dollar,personal_income,year
0,1001,"Autauga, AL",1813553,45151,2020
1,1003,"Baldwin, AL",8762106,51230,2020
2,1005,"Barbour, AL",786529,37111,2020
3,1007,"Bibb, AL",501320,34938,2020
4,1009,"Blount, AL",973414,38133,2020
...,...,...,...,...,...
3095,56037,"Sweetwater, WY",3399440,54708,2020
3096,56039,"Teton, WY",2819873,300665,2020
3097,56041,"Uinta, WY",823225,42289,2020
3098,56043,"Washakie, WY",365071,55098,2020


In [343]:
series_2021_GDP = pd.Series(year2021_GDP, name ='GDP_current_dollar')
first_two_columns['GDP_current_dollar'] = series_2021_GDP

series_2021_PER = pd.Series(year2021_PER, name ='personal_income')
first_two_columns['personal_income'] = series_2021_PER

first_two_columns['year'] = 2021

first_two_columns_2021 = first_two_columns.copy()
first_two_columns_2021

,County,GeoName,GDP_current_dollar,personal_income,year
0,1001,"Autauga, AL",1947622,48914,2021
1,1003,"Baldwin, AL",9879022,55865,2021
2,1005,"Barbour, AL",862519,40795,2021
3,1007,"Bibb, AL",528278,37175,2021
4,1009,"Blount, AL",1153245,42852,2021
...,...,...,...,...,...
3095,56037,"Sweetwater, WY",3814348,56252,2021
3096,56039,"Teton, WY",3947580,362522,2021
3097,56041,"Uinta, WY",913936,44358,2021
3098,56043,"Washakie, WY",391162,54898,2021


In [344]:
series_2022_GDP = pd.Series(year2022_GDP, name ='GDP_current_dollar')
first_two_columns['GDP_current_dollar'] = series_2022_GDP

series_2022_PER = pd.Series(year2022_PER, name ='personal_income')
first_two_columns['personal_income'] = series_2022_PER

first_two_columns['year'] = 2022

first_two_columns_2022 = first_two_columns.copy()
first_two_columns_2022

,County,GeoName,GDP_current_dollar,personal_income,year
0,1001,"Autauga, AL",2364891,49391,2022
1,1003,"Baldwin, AL",10811119,56747,2022
2,1005,"Barbour, AL",930064,40560,2022
3,1007,"Bibb, AL",558848,37513,2022
4,1009,"Blount, AL",1282006,43744,2022
...,...,...,...,...,...
3095,56037,"Sweetwater, WY",4118875,58374,2022
3096,56039,"Teton, WY",4156308,406054,2022
3097,56041,"Uinta, WY",1033918,44775,2022
3098,56043,"Washakie, WY",421190,55288,2022


In [345]:
merged_df34 = pd.concat([first_two_columns_2020, first_two_columns_2021, first_two_columns_2022], ignore_index=True)
merged_df34

,County,GeoName,GDP_current_dollar,personal_income,year
0,1001,"Autauga, AL",1813553,45151,2020
1,1003,"Baldwin, AL",8762106,51230,2020
2,1005,"Barbour, AL",786529,37111,2020
3,1007,"Bibb, AL",501320,34938,2020
4,1009,"Blount, AL",973414,38133,2020
...,...,...,...,...,...
9295,56037,"Sweetwater, WY",4118875,58374,2022
9296,56039,"Teton, WY",4156308,406054,2022
9297,56041,"Uinta, WY",1033918,44775,2022
9298,56043,"Washakie, WY",421190,55288,2022


In [346]:
merged_df34['GeoName'] = merged_df34['GeoName'].str.split(',').str[0]

In [347]:
merged_df34 = merged_df34.rename(columns={'year': 'YEAR'})
merged_df34 = merged_df34.rename(columns={'GeoName': 'CTYNAME'})
merged_df34 = merged_df34.sort_values(by='CTYNAME', ascending=True)

In [348]:
merged_df34['County'] = merged_df34['County'].astype(str)

# Separate into "STATE" and "COUNTY" columns
merged_df34['STATE'] = merged_df34['County'].str[:2]
merged_df34['COUNTY'] = merged_df34['County'].str[2:]

# Convert back to integers if needed
merged_df34['STATE'] = merged_df34['STATE'].astype(int)
merged_df34['COUNTY'] = merged_df34['COUNTY'].astype(int)
merged_df34

,County,CTYNAME,GDP_current_dollar,personal_income,YEAR,STATE,COUNTY
2304,45001,Abbeville,609670,38062,2020,45,1
8504,45001,Abbeville,666910,39840,2022,45,1
5404,45001,Abbeville,608208,40219,2021,45,1
1103,22001,Acadia,1649117,43949,2020,22,1
7303,22001,Acadia,1832352,45671,2022,22,1
...,...,...,...,...,...,...,...
5864,48507,Zavala,698624,42985,2021,48,507
8964,48507,Zavala,928151,43549,2022,48,507
2415,46137,Ziebach,40298,28178,2020,46,137
8615,46137,Ziebach,64011,33700,2022,46,137


In [349]:
merged_df25['CTYNAME'] = merged_df25['CTYNAME'].str.replace(' County', '')
merged_df25 = merged_df25[merged_df25['YEAR'] != 1]

In [350]:
merged_df25['YEAR'] = merged_df25['YEAR'].replace(2, 2020)
merged_df25['YEAR'] = merged_df25['YEAR'].replace(3, 2021)
merged_df25['YEAR'] = merged_df25['YEAR'].replace(4, 2022)
merged_df25 = merged_df25.sort_values(by='CTYNAME', ascending=True)
merged_df25

/var/folders/84/xq_0_7lj3xd98d9n_0cn_m7w0000gn/T/ipykernel_49091/3933743108.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df25['YEAR'] = merged_df25['YEAR'].replace(2, 2020)
/var/folders/84/xq_0_7lj3xd98d9n_0cn_m7w0000gn/T/ipykernel_49091/3933743108.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df25['YEAR'] = merged_df25['YEAR'].replace(3, 2021)
/var/folders/84/xq_0_7lj3xd98d9n_0cn_m7w0000gn/T/ipykernel_49091/3933743108.py:3: SettingWithCopyWarning: 
A value is trying to be set on 

,STATE,COUNTY,STNAME,CTYNAME,YEAR,TOT_POP,TOT_MALE,TOT_FEMALE,WA_MALE,WA_FEMALE,...,AGE7579_FEM,AGE8084_TOT,AGE8084_MALE,AGE8084_FEM,AGE85PLUS_TOT,AGE85PLUS_MALE,AGE85PLUS_FEM,MEDIAN_AGE_TOT,MEDIAN_AGE_MALE,MEDIAN_AGE_FEM
9275,45,1,South Carolina,Abbeville,2022,24356,11851,12505,8613,8798,...,578,660,281,379,564,202,362,44.6,43.2,45.8
9274,45,1,South Carolina,Abbeville,2021,24258,11864,12394,8564,8696,...,542,634,266,368,571,206,365,44.5,43.1,46.0
9273,45,1,South Carolina,Abbeville,2020,24256,11819,12437,8491,8692,...,539,627,259,368,578,206,372,44.6,43.2,46.0
4463,22,1,Louisiana,Acadia Parish,2022,56744,27751,28993,22192,23030,...,967,1023,453,570,1006,307,699,37.5,36.1,39.0
4462,22,1,Louisiana,Acadia Parish,2021,57262,28137,29125,22505,23179,...,899,1055,454,601,1021,317,704,37.4,35.8,38.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11113,48,507,Texas,Zavala,2020,9662,4876,4786,4647,4603,...,141,159,73,86,163,65,98,32.3,31.3,33.3
11115,48,507,Texas,Zavala,2022,9377,4749,4628,4516,4423,...,156,144,59,85,113,56,57,32.5,31.4,33.7
9719,46,137,South Dakota,Ziebach,2022,2395,1213,1182,325,297,...,28,32,16,16,25,10,15,33.1,32.0,33.6
9717,46,137,South Dakota,Ziebach,2020,2392,1200,1192,327,302,...,24,27,13,14,32,16,16,32.8,33.5,31.8


In [351]:
merged_df251 = merged_df25[merged_df25['CTYNAME'] == "Weston"]
merged_df251

,STATE,COUNTY,STNAME,CTYNAME,YEAR,TOT_POP,TOT_MALE,TOT_FEMALE,WA_MALE,WA_FEMALE,...,AGE7579_FEM,AGE8084_TOT,AGE8084_MALE,AGE8084_FEM,AGE85PLUS_TOT,AGE85PLUS_MALE,AGE85PLUS_FEM,MEDIAN_AGE_TOT,MEDIAN_AGE_MALE,MEDIAN_AGE_FEM
12573,56,45,Wyoming,Weston,2020,6818,3724,3094,3470,2876,...,110,171,70,101,193,73,120,43.5,42.6,44.8
12575,56,45,Wyoming,Weston,2022,6860,3800,3060,3532,2837,...,127,141,71,70,170,58,112,44.5,43.6,46.0
12574,56,45,Wyoming,Weston,2021,6766,3729,3037,3468,2822,...,115,151,66,85,184,69,115,43.9,43.1,45.3


In [352]:
merged_df341 = merged_df34[merged_df34['CTYNAME'] == "Weston"]
merged_df341

,County,CTYNAME,GDP_current_dollar,personal_income,YEAR,STATE,COUNTY
9299,56045,Weston,413672,50987,2022,56,45
6199,56045,Weston,324496,49734,2021,56,45
3099,56045,Weston,274529,49854,2020,56,45


In [353]:
merged_df = pd.merge(merged_df25, merged_df34, on=['STATE','COUNTY','YEAR'], how='inner')
columns_to_drop = ['County','CTYNAME_y']
merged_df.drop(columns=columns_to_drop, inplace=True)
merged_df

,STATE,COUNTY,STNAME,CTYNAME_x,YEAR,TOT_POP,TOT_MALE,TOT_FEMALE,WA_MALE,WA_FEMALE,...,AGE8084_MALE,AGE8084_FEM,AGE85PLUS_TOT,AGE85PLUS_MALE,AGE85PLUS_FEM,MEDIAN_AGE_TOT,MEDIAN_AGE_MALE,MEDIAN_AGE_FEM,GDP_current_dollar,personal_income
0,45,1,South Carolina,Abbeville,2022,24356,11851,12505,8613,8798,...,281,379,564,202,362,44.6,43.2,45.8,666910,39840
1,45,1,South Carolina,Abbeville,2021,24258,11864,12394,8564,8696,...,266,368,571,206,365,44.5,43.1,46.0,608208,40219
2,45,1,South Carolina,Abbeville,2020,24256,11819,12437,8491,8692,...,259,368,578,206,372,44.6,43.2,46.0,609670,38062
3,22,1,Louisiana,Acadia Parish,2022,56744,27751,28993,22192,23030,...,453,570,1006,307,699,37.5,36.1,39.0,1832352,45671
4,22,1,Louisiana,Acadia Parish,2022,56744,27751,28993,22192,23030,...,453,570,1006,307,699,37.5,36.1,39.0,(NA),(NA)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8485,48,507,Texas,Zavala,2020,9662,4876,4786,4647,4603,...,73,86,163,65,98,32.3,31.3,33.3,444583,39470
8486,48,507,Texas,Zavala,2022,9377,4749,4628,4516,4423,...,59,85,113,56,57,32.5,31.4,33.7,928151,43549
8487,46,137,South Dakota,Ziebach,2022,2395,1213,1182,325,297,...,16,16,25,10,15,33.1,32.0,33.6,64011,33700
8488,46,137,South Dakota,Ziebach,2020,2392,1200,1192,327,302,...,13,14,32,16,16,32.8,33.5,31.8,40298,28178


In [354]:
merged_df_check1 = merged_df[merged_df['CTYNAME_x'] == "Weston"]
merged_df_check1

,STATE,COUNTY,STNAME,CTYNAME_x,YEAR,TOT_POP,TOT_MALE,TOT_FEMALE,WA_MALE,WA_FEMALE,...,AGE8084_MALE,AGE8084_FEM,AGE85PLUS_TOT,AGE85PLUS_MALE,AGE85PLUS_FEM,MEDIAN_AGE_TOT,MEDIAN_AGE_MALE,MEDIAN_AGE_FEM,GDP_current_dollar,personal_income
8187,56,45,Wyoming,Weston,2020,6818,3724,3094,3470,2876,...,70,101,193,73,120,43.5,42.6,44.8,274529,49854
8188,56,45,Wyoming,Weston,2022,6860,3800,3060,3532,2837,...,71,70,170,58,112,44.5,43.6,46.0,413672,50987
8189,56,45,Wyoming,Weston,2021,6766,3729,3037,3468,2822,...,66,85,184,69,115,43.9,43.1,45.3,324496,49734


In [355]:
merged_df_check2 = merged_df34[merged_df34['CTYNAME'] == "Weston"]
merged_df_check2

,County,CTYNAME,GDP_current_dollar,personal_income,YEAR,STATE,COUNTY
9299,56045,Weston,413672,50987,2022,56,45
6199,56045,Weston,324496,49734,2021,56,45
3099,56045,Weston,274529,49854,2020,56,45


In [356]:
merged_df_check3 = merged_df25[merged_df25['CTYNAME'] == "Weston"]
merged_df_check3

,STATE,COUNTY,STNAME,CTYNAME,YEAR,TOT_POP,TOT_MALE,TOT_FEMALE,WA_MALE,WA_FEMALE,...,AGE7579_FEM,AGE8084_TOT,AGE8084_MALE,AGE8084_FEM,AGE85PLUS_TOT,AGE85PLUS_MALE,AGE85PLUS_FEM,MEDIAN_AGE_TOT,MEDIAN_AGE_MALE,MEDIAN_AGE_FEM
12573,56,45,Wyoming,Weston,2020,6818,3724,3094,3470,2876,...,110,171,70,101,193,73,120,43.5,42.6,44.8
12575,56,45,Wyoming,Weston,2022,6860,3800,3060,3532,2837,...,127,141,71,70,170,58,112,44.5,43.6,46.0
12574,56,45,Wyoming,Weston,2021,6766,3729,3037,3468,2822,...,115,151,66,85,184,69,115,43.9,43.1,45.3


In [357]:
merged_df_2 = pd.merge(df1, merged_df, on=['STATE','COUNTY','YEAR'], how='inner')
columns_to_drop = ['county','fips','state']
merged_df_2.drop(columns=columns_to_drop, inplace=True)

In [358]:
merged_df_2 = merged_df_2.rename(columns={'date': 'Date'})
merged_df_2

,Date,cases,deaths,STATE,COUNTY,YEAR,STNAME,CTYNAME_x,TOT_POP,TOT_MALE,...,AGE8084_MALE,AGE8084_FEM,AGE85PLUS_TOT,AGE85PLUS_MALE,AGE85PLUS_FEM,MEDIAN_AGE_TOT,MEDIAN_AGE_MALE,MEDIAN_AGE_FEM,GDP_current_dollar,personal_income
0,2020-01-21,1,0,53,61,2020,Washington,Snohomish,829869,419770,...,4938,6851,11938,4081,7857,38.3,37.8,38.9,48736536,62805
1,2020-01-22,1,0,53,61,2020,Washington,Snohomish,829869,419770,...,4938,6851,11938,4081,7857,38.3,37.8,38.9,48736536,62805
2,2020-01-23,1,0,53,61,2020,Washington,Snohomish,829869,419770,...,4938,6851,11938,4081,7857,38.3,37.8,38.9,48736536,62805
3,2020-01-24,1,0,53,61,2020,Washington,Snohomish,829869,419770,...,4938,6851,11938,4081,7857,38.3,37.8,38.9,48736536,62805
4,2020-01-25,1,0,53,61,2020,Washington,Snohomish,829869,419770,...,4938,6851,11938,4081,7857,38.3,37.8,38.9,48736536,62805
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
327538,2020-07-22,8,0,31,135,2020,Nebraska,Perkins,2863,1460,...,39,50,108,38,70,42.2,40.6,44.1,317702,73168
327539,2020-07-21,3,0,30,15,2020,Montana,Chouteau,5895,2994,...,66,76,147,61,86,42.3,41.7,43.0,286807,45373
327540,2020-07-22,3,0,30,15,2020,Montana,Chouteau,5895,2994,...,66,76,147,61,86,42.3,41.7,43.0,286807,45373
327541,2020-07-22,1,0,20,203,2020,Kansas,Wichita,2149,1144,...,22,32,75,31,44,39.7,40.7,38.7,189619,60209


In [359]:
df6 = pd.read_csv('COVID-19_Vaccinations_in_the_United_States_County_20240118.csv')
df6.dropna(inplace=True)
df6

/var/folders/84/xq_0_7lj3xd98d9n_0cn_m7w0000gn/T/ipykernel_49091/849764769.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df6 = pd.read_csv('COVID-19_Vaccinations_in_the_United_States_County_20240118.csv')


,Date,FIPS,MMWR_week,Recip_County,Recip_State,Completeness_pct,Administered_Dose1_Recip,Administered_Dose1_Pop_Pct,Administered_Dose1_Recip_5Plus,Administered_Dose1_Recip_5PlusPop_Pct,...,Census2019_18PlusPop,Census2019_65PlusPop,Bivalent_Booster_5Plus,Bivalent_Booster_5Plus_Pop_Pct,Bivalent_Booster_12Plus,Bivalent_Booster_12Plus_Pop_Pct,Bivalent_Booster_18Plus,Bivalent_Booster_18Plus_Pop_Pct,Bivalent_Booster_65Plus,Bivalent_Booster_65Plus_Pop_Pct
0,05/10/2023,55129,19,Washburn County,WI,96.7,11123.0,70.8,11097.0,73.9,...,12758.0,4304.0,3978.0,26.5,3955.0,28.6,3899.0,30.6,2588.0,60.1
1,05/10/2023,19173,19,Taylor County,IA,97.3,3149.0,51.4,3145.0,55.0,...,4687.0,1392.0,841.0,14.7,841.0,16.3,834.0,17.8,588.0,42.2
2,05/10/2023,36059,19,Nassau County,NY,97.5,1391226.0,95.0,1384503.0,95.0,...,1065968.0,246690.0,236202.0,18.4,231748.0,19.8,224931.0,21.1,109947.0,44.6
3,05/10/2023,48281,19,Lampasas County,TX,98.9,11678.0,54.5,11660.0,57.4,...,16799.0,4322.0,1951.0,9.6,1947.0,10.5,1925.0,11.5,1229.0,28.4
5,05/10/2023,13101,19,Echols County,GA,88.8,1573.0,39.3,1572.0,41.4,...,2963.0,553.0,92.0,2.4,88.0,2.7,86.0,2.9,59.0,10.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85352,11/16/2022,27073,46,Lac qui Parle County,MN,94.7,4191.0,63.3,4152.0,65.9,...,5259.0,1847.0,1016.0,16.1,1010.0,17.5,997.0,19.0,679.0,36.8
85353,11/16/2022,36077,46,Otsego County,NY,97.5,41196.0,69.2,40935.0,71.8,...,50138.0,12936.0,7876.0,13.8,7840.0,14.7,7638.0,15.2,4425.0,34.2
85355,11/16/2022,05113,46,Polk County,AR,92.1,9320.0,46.7,9310.0,49.4,...,15424.0,4717.0,630.0,3.3,629.0,3.7,624.0,4.0,448.0,9.5
85357,11/16/2022,46015,46,Brule County,SD,96.7,4522.0,85.4,4494.0,90.9,...,3932.0,1044.0,497.0,10.1,494.0,11.2,482.0,12.3,303.0,29.0


In [360]:
df6['fips'] = df6['FIPS'].astype(str)

# Separate into "STATE" and "COUNTY" columns
df6['STATE'] = df6['FIPS'].str[:2]
df6['COUNTY'] = df6['FIPS'].str[2:]

# Convert back to integers if needed
df6['STATE'] = df6['STATE'].astype(int)
df6['COUNTY'] = df6['COUNTY'].astype(int)
df6

,Date,FIPS,MMWR_week,Recip_County,Recip_State,Completeness_pct,Administered_Dose1_Recip,Administered_Dose1_Pop_Pct,Administered_Dose1_Recip_5Plus,Administered_Dose1_Recip_5PlusPop_Pct,...,Bivalent_Booster_5Plus_Pop_Pct,Bivalent_Booster_12Plus,Bivalent_Booster_12Plus_Pop_Pct,Bivalent_Booster_18Plus,Bivalent_Booster_18Plus_Pop_Pct,Bivalent_Booster_65Plus,Bivalent_Booster_65Plus_Pop_Pct,fips,STATE,COUNTY
0,05/10/2023,55129,19,Washburn County,WI,96.7,11123.0,70.8,11097.0,73.9,...,26.5,3955.0,28.6,3899.0,30.6,2588.0,60.1,55129,55,129
1,05/10/2023,19173,19,Taylor County,IA,97.3,3149.0,51.4,3145.0,55.0,...,14.7,841.0,16.3,834.0,17.8,588.0,42.2,19173,19,173
2,05/10/2023,36059,19,Nassau County,NY,97.5,1391226.0,95.0,1384503.0,95.0,...,18.4,231748.0,19.8,224931.0,21.1,109947.0,44.6,36059,36,59
3,05/10/2023,48281,19,Lampasas County,TX,98.9,11678.0,54.5,11660.0,57.4,...,9.6,1947.0,10.5,1925.0,11.5,1229.0,28.4,48281,48,281
5,05/10/2023,13101,19,Echols County,GA,88.8,1573.0,39.3,1572.0,41.4,...,2.4,88.0,2.7,86.0,2.9,59.0,10.7,13101,13,101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85352,11/16/2022,27073,46,Lac qui Parle County,MN,94.7,4191.0,63.3,4152.0,65.9,...,16.1,1010.0,17.5,997.0,19.0,679.0,36.8,27073,27,73
85353,11/16/2022,36077,46,Otsego County,NY,97.5,41196.0,69.2,40935.0,71.8,...,13.8,7840.0,14.7,7638.0,15.2,4425.0,34.2,36077,36,77
85355,11/16/2022,05113,46,Polk County,AR,92.1,9320.0,46.7,9310.0,49.4,...,3.3,629.0,3.7,624.0,4.0,448.0,9.5,05113,5,113
85357,11/16/2022,46015,46,Brule County,SD,96.7,4522.0,85.4,4494.0,90.9,...,10.1,494.0,11.2,482.0,12.3,303.0,29.0,46015,46,15


In [361]:
df6['Date'] = pd.to_datetime(df1['date'])

In [363]:
merged_df_3 = pd.merge(df6, merged_df_2, on=['STATE','COUNTY','Date'], how='inner')
columns_to_drop = ['FIPS']
merged_df_3.drop(columns=columns_to_drop, inplace=True)
merged_df_3

,Date,MMWR_week,Recip_County,Recip_State,Completeness_pct,Administered_Dose1_Recip,Administered_Dose1_Pop_Pct,Administered_Dose1_Recip_5Plus,Administered_Dose1_Recip_5PlusPop_Pct,Administered_Dose1_Recip_12Plus,...,AGE8084_MALE,AGE8084_FEM,AGE85PLUS_TOT,AGE85PLUS_MALE,AGE85PLUS_FEM,MEDIAN_AGE_TOT,MEDIAN_AGE_MALE,MEDIAN_AGE_FEM,GDP_current_dollar,personal_income
0,2020-02-21,19,Bryan County,OK,93.8,26225.0,54.6,26195.0,58.3,25819.0,...,424,587,912,337,575,37.4,36.0,38.7,1917349,40728
1,2020-02-21,19,Bryan County,OK,93.8,26225.0,54.6,26195.0,58.3,25819.0,...,424,587,912,337,575,37.4,36.0,38.7,280445536,56024
2,2020-02-28,19,Spokane County,WA,96.0,374252.0,71.6,371242.0,75.5,357702.0,...,4199,5498,10228,3528,6700,37.9,36.9,38.9,28473686,49083
3,2020-02-29,19,Dane County,WI,96.7,525490.0,95.0,513813.0,95.0,480996.0,...,3462,4750,9504,3178,6326,35.5,34.7,36.4,46459364,65500
4,2020-03-06,19,Fayette County,KY,94.1,265335.0,82.1,262822.0,86.4,250652.0,...,1964,2979,5249,1659,3590,35.1,34.1,36.2,21244943,55601
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51518,2020-04-24,46,Johnson County,IA,97.4,127666.0,84.5,125343.0,88.0,117738.0,...,808,1138,2398,775,1623,30.6,30.5,30.8,10111237,56435
51519,2020-04-24,46,Calhoun County,GA,89.0,3471.0,56.1,3470.0,58.4,3402.0,...,38,56,143,24,119,40.2,38.8,44.4,129771,32097
51520,2020-04-24,46,Lac qui Parle County,MN,94.7,4191.0,63.3,4152.0,65.9,3988.0,...,118,128,358,131,227,48.8,47.5,50.1,296064,56610
51521,2020-04-24,46,Otsego County,NY,97.5,41196.0,69.2,40935.0,71.8,39355.0,...,643,796,1540,556,984,41.8,41.4,42.2,2773670,50514


In [364]:
columns = merged_df_3.columns.tolist()
columns

['Date',
 'MMWR_week',
 'Recip_County',
 'Recip_State',
 'Completeness_pct',
 'Administered_Dose1_Recip',
 'Administered_Dose1_Pop_Pct',
 'Administered_Dose1_Recip_5Plus',
 'Administered_Dose1_Recip_5PlusPop_Pct',
 'Administered_Dose1_Recip_12Plus',
 'Administered_Dose1_Recip_12PlusPop_Pct',
 'Administered_Dose1_Recip_18Plus',
 'Administered_Dose1_Recip_18PlusPop_Pct',
 'Administered_Dose1_Recip_65Plus',
 'Administered_Dose1_Recip_65PlusPop_Pct',
 'Series_Complete_Yes',
 'Series_Complete_Pop_Pct',
 'Series_Complete_5Plus',
 'Series_Complete_5PlusPop_Pct',
 'Series_Complete_5to17',
 'Series_Complete_5to17Pop_Pct',
 'Series_Complete_12Plus',
 'Series_Complete_12PlusPop_Pct',
 'Series_Complete_18Plus',
 'Series_Complete_18PlusPop_Pct',
 'Series_Complete_65Plus',
 'Series_Complete_65PlusPop_Pct',
 'Booster_Doses',
 'Booster_Doses_Vax_Pct',
 'Booster_Doses_5Plus',
 'Booster_Doses_5Plus_Vax_Pct',
 'Booster_Doses_12Plus',
 'Booster_Doses_12Plus_Vax_Pct',
 'Booster_Doses_18Plus',
 'Booster_Dos

In [372]:
file_path = '/Users/asherbank/Desktop/merged_df_ashv2.csv'
merged_df_3.to_csv(file_path, index = False)